In [108]:
import pandas as pd


### 紐付きデータ

In [109]:
himotuki_path = "CSV1_先頭40件.csv"
df_himotuki = pd.read_csv(himotuki_path, encoding="utf-8")
df_himotuki.head(5)

,syutugan,himotuki,category,koukaibi,syutugan_path（クエリ）,himotuki_path（ナレッジ）
0,JP2013224028A,JP2012040876A,Ax,20120301,result_4/28/JP2013224028A/text.txt,result_3/12/JP2012040876A/text.txt
1,JP2014007731A,JP2011171723A,Ax,20110901,result_4/24/JP2014007731A/text.txt,result_2/30/JP2011171723A/text.txt
2,JP2014037831A,JP2012145109A,Ax,20120802,result_5/2/JP2014037831A/text.txt,result_3/24/JP2012145109A/text.txt
3,JP2014038623A,JP2012502395A,Ax,20120126,result_5/2/JP2014038623A/text.txt,result_3/8/JP2012502395A/text.txt
4,JP2014062952A,JP2011070088A,Ax,20110407,result_4/11/JP2014062952A/text.txt,result_1/31/JP2011070088A/text.txt


### RAG予測結果

In [110]:
rag_output_path = "rag_output.csv"
df_rag = pd.read_csv(rag_output_path)
df_rag

,query_id,knowledge_id
0,2014062952,2011070088
1,2014062952,2011070088
2,2014062952,2011128587
3,2014007731,2011171723
4,2014007731,2011171723
5,2014007731,2011139055
6,2013224028,2012040876
7,2013224028,2012041441
8,2013224028,2012041441


In [111]:
# 紐付きCSV
syutugan_ids = df_himotuki["syutugan"].tolist()
himotuki_ids = df_himotuki["himotuki"].tolist()

# RAG結果
query_ids = df_rag["query_id"].tolist()
knowledge_ids = df_rag["knowledge_id"].tolist()

is_correct_list = []
for i in range(len(query_ids)):
    query_id = str(query_ids[i])
    knowledge_id = str(knowledge_ids[i])

    # クエリに紐づくID（正解）を探す
    himotuki_id = None
    for i, syutugan_id in enumerate(syutugan_ids):
        if query_id in syutugan_id:
            himotuki_id = str(himotuki_ids[i])
            break

    if himotuki_id is None:
        raise ValueError("紐づく文献が見つかりませんでした。")

    if knowledge_id in himotuki_id:  # 例: "2023034567" in "JP2023034567A" -> True
        is_correct = True
    else:
        is_correct = False

    is_correct_list.append(is_correct)

print(is_correct_list)

[True, True, False, True, True, False, True, False, False]


In [112]:
# 評価結果をcsv出力
df_rag["is_correct"] = is_correct_list
df_rag.to_csv("rag_output_with_TF.csv", index=False, encoding="utf-8")